<a href="https://colab.research.google.com/github/ambulesubhada07/Speech_learning/blob/main/Error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

In [ ]:
import json
import requests
import pandas as pd
import gradio as gr
import librosa
import soundfile as sf
import torch
import warnings #import necessary modul

In [ ]:
with open('/content/HF_SECRET_KEY (1) (1).json') as f:
    hf_data = json.load(f)

In [ ]:
API_TOKEN = hf_data["API_KEY"]


In [ ]:
#ASR_API_URL = "https://api-inference.huggingface.co/models/facebook/wav2vec2-base-960h"
ASR_API_URL = "https://api-inference.huggingface.co/models/openai/whisper-large-v3"
#Q_A_API_URL = "https://api-inference.huggingface.co/models/google/tapas-base-finetuned-wtq"
Q_A_API_URL = "https://api-inference.huggingface.co/models/google/tapas-large-finetuned-wtq"

In [ ]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}


In [ ]:
def q_n_a_query(payload):

    data = json.dumps(payload)
    response = requests.request("POST",
                                Q_A_API_URL,
                                headers=headers,
                                data=data)
    return json.loads(response.content.decode("utf-8"))

In [ ]:
def asr_query(audio_file):

    if audio_file is None: # if audio file is empty then this code excecute
     gr.Warning('Audio file is emtpy')
     return table,'',''
    else:  # if audio file is there then this code  will excecute
     gr.Info("Starting process")
     sr,data =audio_file
     sf.write("temp.wav", data, 44100)
     with open("temp.wav", "rb") as f:
        data = f.read()
     response = requests.request("POST",  # hit the asr pretend model
                                ASR_API_URL,
                                headers=headers,
                                data=data) # this is api that use for to convert audio to text

     output = json.loads(response.content.decode("utf-8"))
     table_query = output["text"]
     data = q_n_a_query(payload={ # call q_n_a_query function with parameters
            "inputs": {
            "query": table_query,
            "table": table.to_dict('list')}})
     if "text" in table_query: #Ensure key exists in dictionary
       gr.Warning('please try again later')
       return table, '', ''
     else:
       if "answer" in  data["answer"]: #Ensure key exists in dictionary
        gr.Warning('please try again later a')
        return table, '', ''
       else:
        if  data["answer"]=="": # if somthing we spoke and that data is not there then this condition will excecute
         gr.Warning('data is not Found')
         return table, table_query, ''
        else:
         return table, table_query, data["answer"]

In [ ]:
table = pd.read_csv("data (1).csv",low_memory=False)


In [ ]:
table.head(3)


,Pos,Player,Team,Span,Innings,Runs,Highest Score,Average,Strike Rate
0,1,Sachin Tendulkar,India,1989-2012,452,18426,200,44.83,86.23
1,2,Kumar Sangakkara,Sri Lanka,2000-2015,380,14234,169,41.98,78.86
2,3,Ricky Ponting,Australia,1995-2012,365,13704,164,42.03,80.39


In [ ]:
table

,Pos,Player,Team,Span,Innings,Runs,Highest Score,Average,Strike Rate
0,1,Sachin Tendulkar,India,1989-2012,452,18426,200,44.83,86.23
1,2,Kumar Sangakkara,Sri Lanka,2000-2015,380,14234,169,41.98,78.86
2,3,Ricky Ponting,Australia,1995-2012,365,13704,164,42.03,80.39
3,4,Sanath Jayasuriya,Sri Lanka,1989-2011,433,13430,189,32.36,91.20
4,5,Mahela Jayawardene,Sri Lanka,1998-2015,418,12650,144,33.37,78.96
5,6,Virat Kohli,India,2008-2020,236,11867,183,59.85,93.39
6,7,Inzamam-ul-Haq,Pakistan,1991-2007,350,11739,137,39.52,74.24
7,8,Jacques Kallis,South Africa,1996-2014,314,11579,139,44.36,72.89
8,9,Saurav Ganguly,India,1992-2007,300,11363,183,41.02,73.70
9,10,Rahul Dravid,India,1996-2011,318,10889,153,39.16,71.24


In [ ]:
table = table.astype(str)


In [ ]:
iface = gr.Interface(asr_query, "microphone", ["dataframe",
                                               "text",
                                               "text"])
iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://698b67f92b7d622974.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2108, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1655, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^